# Úkol č. 2 - předzpracování dat a binární klasifikace (do 9. listopadu 23:59)

  * V rámci tohoto úkolu se musíte vypořádat s příznaky, které jsou různých typů.
  * Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na podmnožiny vhodné k trénování modelu.
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.
  * Ukázka prvních řádků souboru *results.csv*:
  
```
ID,survived
1000,0
1001,1
...
```

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí křížové validace. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor s predikcemi pro vyhodnocovací data (`results.csv`).
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, ParameterGrid, KFold
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib
import matplotlib.pyplot as plt

### Načtení dat ze souboru, jejich úprava a rozdělení

In [2]:
# 1. nactete ze souboru data
training_data = pd.read_csv('data.csv')

In [3]:
display(training_data.shape)
display(training_data.head())
display(training_data.info())
display(training_data.describe())
display(training_data.nunique())

(1000, 13)

,ID,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
0,0,1,3,"Dorking, Mr. Edward Arthur",male,19.0,0,0,A/5. 10482,8.0500,NaN,S,"England Oglesby, IL"
1,1,1,2,"Smith, Miss. Marion Elsie",female,40.0,0,0,31418,13.0000,NaN,S,NaN
2,2,0,3,"Hegarty, Miss. Hanora ""Nora""",female,18.0,0,0,365226,6.7500,NaN,Q,NaN
3,3,0,3,"Sage, Mr. John George",male,NaN,1,9,CA. 2343,69.5500,NaN,S,NaN
4,4,0,3,"Cacic, Miss. Marija",female,30.0,0,0,315084,8.6625,NaN,S,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         1000 non-null   int64  
 1   survived   1000 non-null   int64  
 2   pclass     1000 non-null   int64  
 3   name       1000 non-null   object 
 4   sex        1000 non-null   object 
 5   age        797 non-null    float64
 6   sibsp      1000 non-null   int64  
 7   parch      1000 non-null   int64  
 8   ticket     1000 non-null   object 
 9   fare       1000 non-null   float64
 10  cabin      226 non-null    object 
 11  embarked   998 non-null    object 
 12  home.dest  554 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 101.7+ KB


None

,ID,survived,pclass,age,sibsp,parch,fare
count,1000.000000,1000.000000,1000.000000,797.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,0.393000,2.312000,29.342535,0.503000,0.396000,33.340479
std,288.819436,0.488661,0.832676,14.408116,1.029102,0.861351,50.576026
min,0.000000,0.000000,1.000000,0.333300,0.000000,0.000000,0.000000
25%,249.750000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,499.500000,0.000000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,749.250000,1.000000,3.000000,38.000000,1.000000,0.000000,30.771850
max,999.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


ID           1000
survived        2
pclass          3
name          999
sex             2
age            93
sibsp           7
parch           8
ticket        740
fare          248
cabin         156
embarked        3
home.dest     308
dtype: int64

In [4]:
training_data.isnull().sum(axis=0)

ID             0
survived       0
pclass         0
name           0
sex            0
age          203
sibsp          0
parch          0
ticket         0
fare           0
cabin        774
embarked       2
home.dest    446
dtype: int64

Dropuji sloupce s příznaky, které nechci použít - cabin a home.dest, nepřijdou mi důležité a chybí jich příliš mnoho. Lepších výsledků jsem také dosahovala, když jsem provedla drop sloupce age, kde také chybí mnoho hodnot. Stejně tak jsem provedla drop sloupce name, který má 999 unikátních hodnot a není relevantní, a sloupce fare.

In [5]:
data2 = training_data.drop(columns=['cabin','home.dest', 'age', 'name', 'fare'])

Nahrazuji data formátu object (string) a float.

In [6]:
# Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu
cols_to_replace = data2.select_dtypes(['object']).columns
data2[cols_to_replace] = data2[cols_to_replace].astype('category').apply(lambda x: x.cat.codes)

cols_to_replace2 = data2.select_dtypes(['float64']).columns
data2[cols_to_replace2] = data2[cols_to_replace2].astype('category').apply(lambda x: x.cat.codes)

display(data2.dtypes)

ID          int64
survived    int64
pclass      int64
sex          int8
sibsp       int64
parch       int64
ticket      int16
embarked     int8
dtype: object

In [22]:
# zvlast si dam vysvetlovanou promennou
Xdata = data2.iloc[:, data2.columns != 'survived']
Ydata = data2.iloc[:, data2.columns == 'survived']

In [23]:
# rozdeleni na trenovaci, testovaci a validacni data
rd_seed = 333
Xtrain, Xtest, Ytrain, Ytest = train_test_split(Xdata, Ydata, test_size=0.4, random_state=rd_seed)

U všech následujících modelů jsem provedla křížovou validaci.

### 1. Rozhodovací strom

In [42]:
# 1. naladime parametry a zkousime, ktere hodnoty budou nejlepsi
param_grid = {
    'max_depth': range(1, 51),
    'criterion': ['entropy', 'gini']
}
param_combinations = ParameterGrid(param_grid)
k_folds = 20

In [43]:
for parameters in param_combinations:
    comb_acc = []
    # 1. rozdeleni na trenovaci a validacni
    for train, val in KFold(n_splits = k_folds, random_state = rd_seed, shuffle = True).split(data2.index):
        Xtrain = data2[data2.index.isin(train)].drop(columns='survived')
        Ytrain = data2[data2.index.isin(train)]['survived']
        Xval = data2[data2.index.isin(val)].drop(columns='survived')
        Yval = data2[data2.index.isin(val)]['survived']
        
        # 2. trenink modelu
        model = DecisionTreeClassifier(**parameters)
        model.fit(Xtrain, Ytrain)
        # 3. validacni chyba
        comb_acc.append(metrics.accuracy_score(Yval, model.predict(Xval)))



In [44]:
best_params = param_combinations[np.argmax(comb_acc)]
print("Best parameters: " + str(best_params))
print("Best accuracy: " + str(comb_acc[np.argmax(comb_acc)]))

Best parameters: {'max_depth': 14, 'criterion': 'entropy'}
Best accuracy: 0.82


### 2. Random Forest

In [12]:
param_grid2 = {
    'n_estimators': range(1, 100, 5), # pocet stromu v lese
    'max_depth': range(1, 10)
}
param_combinations2 = ParameterGrid(param_grid2)
k_folds = 5

In [13]:
for parameters in param_combinations2:
    comb_acc2 = []
    # 1. rozdeleni na trenovaci a validacni
    for train, val in KFold(n_splits = k_folds, random_state = rd_seed, shuffle = True).split(data2.index):
        Xtrain = data2[data2.index.isin(train)].drop(columns='survived')
        Ytrain = data2[data2.index.isin(train)]['survived']
        Xval = data2[data2.index.isin(val)].drop(columns='survived')
        Yval = data2[data2.index.isin(val)]['survived']
        # 2. trenink modelu
        model2 = RandomForestClassifier(**parameters)
        model2.fit(Xtrain, Ytrain.values.ravel())
        # 3. validacni chyba
        comb_acc2.append(metrics.accuracy_score(Yval, model2.predict(Xval)))

In [14]:
best_params2 = param_combinations2[np.argmax(comb_acc2)]
print("Best parameters: " + str(best_params2))
print("Best accuracy: " + str(comb_acc2[np.argmax(comb_acc2)]))

Best parameters: {'n_estimators': 11, 'max_depth': 1}
Best accuracy: 0.83


### 3. AdaBoost

In [15]:
param_grid3 = {
    'n_estimators': range(1,100,5),
    'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.75, 1]
}
param_combinations3 = ParameterGrid(param_grid3)
k_folds = 5

In [16]:
for parameters in param_combinations3:
    comb_acc3 = []
    # 1. rozdeleni na trenovaci a validacni
    for train, val in KFold(n_splits = k_folds, random_state = rd_seed, shuffle = True).split(data2.index):
        Xtrain = data2[data2.index.isin(train)].drop(columns='survived')
        Ytrain = data2[data2.index.isin(train)]['survived']
        Xval = data2[data2.index.isin(val)].drop(columns='survived')
        Yval = data2[data2.index.isin(val)]['survived']
        # 2. trenink modelu
        model3 = AdaBoostClassifier(**parameters)
        model3.fit(Xtrain, Ytrain.values.ravel())
        # 3. validacni chyba
        comb_acc3.append(metrics.accuracy_score(Yval, model3.predict(Xval)))

In [17]:
best_params3 = param_combinations3[np.argmax(comb_acc3)]
print("Best parameters: " + str(best_params3))
print("Best accuracy: " + str(comb_acc[np.argmax(comb_acc3)]))

Best parameters: {'n_estimators': 16, 'learning_rate': 0.01}
Best accuracy: 0.7


### 4. kNN

In [18]:
param_grid = {
    'n_neighbors' : range(1, 12),
    'p': range(1, 3),
    'weights': ['uniform', 'distance'],
}
param_combinations4 = ParameterGrid(param_grid)
k_folds = 5

In [71]:
for parameters in param_combinations4:
    comb_acc4 = []
    # 1. rozdeleni na trenovaci a validacni
    for train, val in KFold(n_splits = k_folds, random_state = rd_seed, shuffle = True).split(data2.index):
        Xtrain = data2[data2.index.isin(train)].drop(columns='survived')
        Ytrain = data2[data2.index.isin(train)]['survived']
        Xval = data2[data2.index.isin(val)].drop(columns='survived')
        Yval = data2[data2.index.isin(val)]['survived']
        
        scaler = MinMaxScaler()
        Xtrain = pd.DataFrame(scaler.fit_transform(Xtrain), index=Xtrain.index, columns=Xtrain.columns)
        Xval = pd.DataFrame(scaler.transform(Xval), index=Xval.index, columns=Xval.columns)

        # 2. trenink modelu
        model4 = KNeighborsClassifier(**parameters)
        model4.fit(Xtrain, Ytrain.values.ravel())
        # 3. validacni chyba
        comb_acc4.append(metrics.accuracy_score(Yval, model4.predict(Xval)))

In [72]:
best_params4 = param_combinations4[np.argmax(comb_acc4)]
print("Best parameters: " + str(best_params4))
print("Best accuracy: " + str(comb_acc4[np.argmax(comb_acc4)]))

Best parameters: {'weights': 'uniform', 'p': 1, 'n_neighbors': 3}
Best accuracy: 0.92


### Výběr nejlepšího modelu
Na základě nejlepších přesností na validační množině jsem za nejlepší metodu určila kNN. Nyní tedy provedu trénink  a měření přesnosti tohoto modelu na testovací množině.

In [75]:
k_neighbors = KNeighborsClassifier(**best_params4)
k_neighbors.fit(Xtrain, Ytrain.values.ravel())
print("Accuracy on testing data: " + str(metrics.accuracy_score(Ytest, k_neighbors.predict(Xtest))))

Accuracy on testing data: 0.61


### Predikce přežití na souboru evaluation.csv
Pro predikci tedy použiji kNN a nejlepší parametry, které jsem získala.

In [25]:
# data neobsahuji vysvetlovanou promennou - sloupec 'survived'
evaluating_data = pd.read_csv('evaluation.csv')
final_data = evaluating_data.drop(columns=['cabin','home.dest', 'age', 'name', 'fare'])
display(final_data)
display(final_data.dtypes)

,ID,pclass,sex,sibsp,parch,ticket,embarked
0,1000,2,female,2,1,243847,S
1,1001,2,female,1,1,237789,S
2,1002,2,male,1,0,28664,S
3,1003,3,female,1,0,376566,S
4,1004,2,female,1,0,24065,S
...,...,...,...,...,...,...,...
304,1304,3,female,0,0,329944,Q
305,1305,3,male,0,0,1601,S
306,1306,3,male,0,0,2698,C
307,1307,1,female,1,1,36928,S


ID           int64
pclass       int64
sex         object
sibsp        int64
parch        int64
ticket      object
embarked    object
dtype: object

In [26]:
cols_to_replace = final_data.select_dtypes(['object']).columns
final_data[cols_to_replace] = final_data[cols_to_replace].astype('category').apply(lambda x: x.cat.codes)

In [27]:
X_final = final_data.iloc[:, :]

In [77]:
final_prediction = k_neighbors.predict(X_final)
d = {'ID': X_final['ID'], 'survived': final_prediction}
prediction = pd.DataFrame(data=d).to_csv('results.csv', index=False)